<a href="https://colab.research.google.com/github/tugangui/caseTracker/blob/main/casetracker2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import json
from collections import OrderedDict

# Mount the drive
from google.colab import drive
drive.mount('/content/drive')

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
output_file = './u19_salk_parsed.json'

In [ ]:
# Regex
ID_PATTERN = re.compile(r'[A-Z]{2}[0-9]{6}-[0-9]{2}')
LOCATION_RE = re.compile(r'^[0-9]?\.?\s?([\w +/_\.\'-]+\b)')
COORDINATES_RE = re.compile(r'(\((\+?\-?\d+\.?\d*\,? ?)+\))')
TRACER_RE = re.compile(r'^[0-9]?\.?\s?([\w +/_\.\'-]+\b)')

# Values and substitutions
TECHS = ('ru', 'lei', 'lin', 'marlene', 'stanley', 'sarvia', 'nick', 'danny',
         'fernando')
TECHS_ALT = {'sa': 'sarvia'}
THICKNESS = ('50',)
PLANES = ('C',)
PLANES_ALT = {'coronal': 'C', 'c': 'C'}
STRATEGIES = ('TRIO', 'quadruple retrograde', 'triple anterograde',)
STRATEGIES_ALT = {'trio': 'TRIO',
                  'quadruple retrograde': 'quadruple retrograde',
                  'triple anterograde': 'triple anterograde',
                  'quad': 'quadruple retrograde',
                  'triple ant': 'triple anterograde'}

U01_COLS = ("count", "case", "strategy", "tracer", "target", "target2", "adjusted", "actual", 
               "quality", "surgeon", "perfusion_date", "notes", "plane", "thickness", 
               "sectioning_date", "sectioning_tech", "rescan", "immunostained_for", 
               "staining_date", "staining_tech", "mounting_date", "mounting_tech", "img_process",
              "vsi_path", "osp_path", "image_tech", "dimension", "qa_by")

CSHL_COLS = ("count", "case", "strategy", "tracer", "target", "adjusted", "actual", "quality",
          "surgeon", "perfusion_date", "notes", "plane", "thickness", "sectioning_date",
          "sectioning_tech", "rescan", "immunostained_for", "staining_date",
          "staining_tech", "mounting_date", "mounting_tech", "img_process",
          "vsi_path", "osp_path", "image_tech", "dimension", "registration", "threshold",
          "overlap", "qa_by")

U19_COLS = ("count", "case", "strategy", "tracer", "target", "actual", "quality",
          "surgeon", "perfusion_date", "notes", "empty", "plane", "thickness", "sectioning_date",
          "sectioning_tech", "rescan", "immunostained_for", "staining_date",
          "staining_tech", "mounting_date", "mounting_tech", "img_process",
          "vsi_path", "osp_path", "image_tech", "dimension", "registration", "threshold",
          "overlap", "qa_by")

MCP_COLS = ("count", "case", "strategy", "tracer", "target", "adjusted", "actual", "quality",
          "surgeon", "perfusion_date", "notes", "plane", "thickness", "sectioning_date",
          "sectioning_tech", "immunostained_for", "staining_date",
          "staining_tech", "mounting_date", "mounting_tech", "img_process", "location",
          "vsi_path", "osp_path", "image_tech", "dimension", "registration", "threshold",
          "overlap", "qa_by")

BG_COLS = ("count", "case", "strategy", "tracer", "target", "actual", "quality",
          "surgeon", "perfusion_date", "notes", "plane", "thickness", "sectioning_date",
          "sectioning_tech", "immunostained_for", "staining_date",
          "staining_tech", "mounting_date", "mounting_tech", "microscopy", 
          "vsi_path", "osp_path", "image_tech", "dimension", "registration", "threshold",
          "overlap", "qa_by")

RF1_COLS = ("count", "case", "strategy", "tracer", "target", "actual", "quality",
          "surgeon", "perfusion_date", "notes", "plane", "thickness", "sectioning_date",
          "sectioning_tech", "rescan", "immunostained_for", "staining_date",
          "staining_tech", "mounting_date", "mounting_tech", "img_process",
          "vsi_path", "osp_path", "image_tech", "dimension", "registration", "threshold",
          "overlap", "qa_by")

BLA_COLS = ("count", "case", "strategy", "tracer", "desired", "target", "actual", "quality",
          "surgeon", "perfusion_date", "notes", "plane", "thickness", "sectioning_date",
          "sectioning_tech", "immunostained_for", "staining_date",
          "staining_tech", "mounting_date", "mounting_tech", "img_process",
          "vsi_path", "osp_path", "image_tech", "dimension", "registration", "threshold",
          "overlap", "qa_by")

COLUMNS = {'U19 CSHL': CSHL_COLS, 'U19 Salk Institute': U19_COLS, 'MCP': MCP_COLS, 
           'Basal Ganglia': BG_COLS, 'RF1 (HPF)': RF1_COLS, 'BLA project': BLA_COLS}

SKIPROWS = {'U19 CSHL': 8, 'U19 Salk Institute': 6, 'MCP': 7,
            'Basal Ganglia': 6, 'RF1 (HPF)': 13, 'BLA project': 4}


def prepare_df(df):
    '''
        Moves non case id and strategy vals to notes columns, foward fills case and strategy columns
        fills na values as empty strings
        
        assumes no empty cols or rows, and that case_id and strategy name are both on the 
        first row of case rows at least
        ex.
         x  a b c d e f
        c1 s1 # # # # #
              # # # # #
              # # # # #
        c2 s2 @ @ @ @ @ 
              @ @ @ @ @
              @ @ @ @ @
           
        
    '''
    df = df.copy()
    
    if 'count' in df.columns:
        df.drop(columns=['count'], inplace=True)
    
    df['case_col_notes'] = df['case'][~df['case'].str.match(r'[A-Z]{2}[0-9]{6}-[0-9]{2}', na=False)]
    df.loc[~df['case'].str.match(r'[A-Z]{2}[0-9]{6}-[0-9]{2}', na=False), ['case']] = np.nan
    df['case'] = df[['case']].fillna(method='ffill')

    df['trio_col_notes'] = df['strategy'][~df['strategy'].str.match('TRIO', na=False)]
    df.loc[~df['strategy'].str.match('TRIO', na=False), ['strategy']] = np.nan
    df['strategy'] = df[['strategy']].fillna(method='ffill')
    
    df.fillna('', inplace=True)
    
    return df
    

def get_project_dict(df, project):
    for case_id, data in df.iterrows():
        case_id = case_id[0:11]
        # if first row with this case id
        if case_id not in project:
            project[case_id] = {
                'case_name': case_id,
                'notes': data["notes"],
                'surgery': {
                    'planned_for': get_tech(data["surgeon"]),
                    'surgeon': get_tech(data["surgeon"]),
                    'perfusion_date': get_date_or_none(data["perfusion_date"]),
                    'strategy': data["strategy"],
                    'injections': [
                        {
                            "target_loc": get_location(str(data['target'])),
                            "target_coords": get_coordinates(str(data['target'])),
                            "actual_loc": get_location(str(data['actual'])),
                            "actual_coords": get_coordinates(str(data['actual'])),
                            "adjust_loc": "",
                            "adjust_coords": "",
                            "quality": data['quality'],
                            "tracer": {
                                "name": get_tracer(data["tracer"]),
                                "notes": "",
                            }
                        }
                    ]
                },
                'section': {
                    "plane": get_plane(data["plane"]),
                    "thickness": data["thickness"],
                    "sectioning_date": get_date_or_none(data["sectioning_date"]),
                    "sectioning_tech": get_tech(data["sectioning_tech"])
                },
                "stain": {
                    "staining_date": get_date_or_none(data["staining_date"]),
                    "staining_tech": get_tech(data["staining_tech"]),
                    "immunostained_for": data["immunostained_for"]
                },
                "mount": {
                    "mounting_date": get_date_or_none(data["mounting_date"]),
                    "mounting_tech": get_tech(data["mounting_tech"]),
                },
                "image_process": {
                    "vsi_path": data["vsi_path"],
                    "osp_path": data["osp_path"],
                    "image_tech": get_tech(data["image_tech"]),
                    "dimension": data["dimension"],
                    "qa_by": get_tech(data["qa_by"])
                },
                "scan": {
                    "quality": "",
                }
            }
            case_ = project[case_id]

            add_to_notes(case_, data["case_col_notes"])
            add_to_notes(case_, data["trio_col_notes"])

        else:

            case_ = project[case_id]
            case_['surgery']['injections'].append({
                "target_loc": get_location(str(data['target'])),
                "target_coords": get_coordinates(str(data['target'])),
                "actual_loc": get_location(str(data['actual'])),
                "actual_coords": get_coordinates(str(data['actual'])),
                "adjust_loc": "",
                "adjust_coords": "",
                "quality": data['quality'],
                "tracer": {
                    "name": get_tracer(data["tracer"]),
                    "notes": "",
                }
            })
            if data['surgeon'] != '': #surgeons are often on second row (why) and i'm too lazy to fix it
              case_['surgery']['surgeon'] = data['surgeon']

            for prop in ["surgeon", "perfusion_date", "notes", "plane", "thickness", 
                         "sectioning_date","sectioning_tech", "immunostained_for", 
                         "staining_date", "staining_tech", "mounting_date", "mounting_tech", 
                         "vsi_path", "osp_path", "image_tech", "dimension", 
                          "qa_by", "case_col_notes", 
                         "trio_col_notes"]:

                if prop == 'notes':
                    add_to_notes(case_, data[prop])
                else:
                    add_to_notes(case_, data[prop], prefix=prop)
    
    return project


def get_date_or_none(date_string):
    date_comp = date_string.split('/')
    if len(date_comp) == 3:
        return '-'.join([date_comp[2], date_comp[0], date_comp[1]])
    else:
        return None

def get_location(data):
    loc = LOCATION_RE.search(data) 
    return loc.groups(0)[0] if loc else ''
    
def get_coordinates(data):
    coords = COORDINATES_RE.search(data)
    return coords.groups(0)[0] if coords else ''

def get_tracer(data):
    tracer = TRACER_RE.search(data)
    return tracer.groups(0)[0] if tracer else None

def get_tech(data):
    tech = data.lower()
    if tech in TECHS:
        return tech
    elif tech in TECHS_ALT:
        return TECHS_ALT[tech]
    else:
        return tech

def get_plane(plane):
    if plane in PLANES:
        return plane
    elif plane in PLANES_ALT:
        return PLANES_ALT[plane]

def add_to_notes(case_dictionary, note, prefix=None):
    note = str(note)
    if len(note) > 0:
        if prefix:
            note = "{}: {}".format(prefix, note)
        
        case_dictionary['notes'] += "\n" + note

def save_json(item):
    with open(output_file, 'w') as f:
        f.write(json.dumps(item, indent=4))

def process(df, name, project, save=False):
    cols = COLUMNS[name]
    df.columns = cols
    prepped_df = prepare_df(df)
    good_df = prepped_df.set_index(['case'])
    good_df.to_csv('/good_csv.csv')
    project_dict = get_project_dict(good_df, project)
    
    if save:
        save_json(project_dict)

    return project_dict

In [ ]:
def get_organism_id(argument):
    switcher = {
        'SG': 7,
        'SP': 17,
        'SV': 18
    }
    return switcher.get(argument, 1)
  
#Lin, Lei, Marlene, Kevin
def get_usernames(argument):
    switcher = {
        'monica': "Monica Song",
        'lei': "Lei Gao",
        'marlene': "Marlene Becerra",
        'nick': "Nick Foster",
    }
    return switcher.get(argument, "Lin Gou")

In [ ]:
TRACKERS = {}
for name, col in COLUMNS.items():
  print(name)
  worksheet = gc.open('Copy of Hongwei Dong lab case tracker.xlsx').worksheet(name)
  TRACKERS[name] = pd.DataFrame.from_records(worksheet.get_all_values())

project = {}
for name, tracker in TRACKERS.items():
  tracker_df = tracker
  tracker_df = tracker_df.iloc[SKIPROWS[name]:]
  project = process(tracker_df, name, project, save=True)

save_json(project)

In [ ]:
f = open("/casetracker.sql", "w")

query = "INSERT IGNORE INTO organisms (id, species, strain, allele_type, gene_marker, description, code) VALUES (17, 'mouse', 'Parvalbumin-Cre', 'transgenic', 'Parvalbumin (PV)', 'These mice selectively express Cre recombinase in PV expressing cells', 'P');\n"
query = query + "INSERT IGNORE INTO organisms (id, species, strain, allele_type, gene_marker, description, code) VALUES (18, 'mouse', 'Vasoactive Intestinal Peptide-Cre', 'transgenic', 'Vasoactive Intestinal Peptide (VIP)', 'These mice selectively express Cre recombinase in VIP expressing cells', 'P');\n"
query = query + "INSERT INTO users(user_name, email, user_group, is_active) VALUES('Lei Gao', 'LeiGao@mednet.ucla.edu',4, 1 )ON DUPLICATE KEY UPDATE email='LeiGao@mednet.ucla.edu';\n"
query = query + "create trigger after_injectionSites_insert after insert on injectionSites for each row insert into injectionSitesLocations(injection_sites_id) values(new.id);"

for id, info in project.items():
  case_prefix = get_organism_id(id[:2])
  query = query + "INSERT INTO animals (organism_id) SELECT " + case_prefix + " FROM tissueDissections WHERE NOT EXISTS(SELECT NULL FROM tissueDissections td WHERE td.tissue_code='" + id + "') LIMIT 1;\n"
  query = query + "INSERT IGNORE INTO tissueDissections (tissue_code, tissue_name, description, animal_id) VALUES ('" + id +"', 'brain', 'brain', LAST_INSERT_ID());\n"

for id, info in project.items():
    username = get_usernames(info['surgery']['surgeon'].lower())
    query = query + "\nINSERT INTO surgeries (user_id, animal_id, tissue_dissection_id) SELECT (select id from users where user_name='" + username + "') as user_id, (select animal_id from tissueDissections where tissue_code='" + id + "') as animal_id, (select id FROM tissueDissections WHERE tissue_code='" + id + "') as tissue_code from surgeries WHERE NOT EXISTS(SELECT s.tissue_dissection_id FROM surgeries s INNER JOIN tissueDissections td on s.tissue_dissection_id=td.id WHERE td.tissue_code='" + id + "') LIMIT 1;"

pattern = r'[\(\)\+ ,]'
for id, info in project.items():
  injections = info['surgery']['injections']

  query = query + "\nINSERT INTO injectionSites (surgery_id) SELECT (SELECT surgeries.id FROM surgeries INNER JOIN tissueDissections td ON surgeries.tissue_dissection_id=td.id WHERE td.tissue_code='" + id + "') AS surgery_id FROM injectionSites WHERE NOT EXISTS(SELECT i.surgery_id FROM injectionSites i INNER JOIN surgeries s ON i.surgery_id=s.id INNER JOIN tissueDissections td on s.tissue_dissection_id=td.id WHERE td.tissue_code='" + id + "') LIMIT " + str(len(injections)) + ";" 
  
  ind = len(injections) - 1
  for injection in injections:
    if injection['target_coords'] != '':
        i = injection['target_coords'].split(",")
        target_x = re.sub(pattern, '', i[0])
        target_y = re.sub(pattern, '', i[1])
        target_z = re.sub(pattern, '', i[2])
    else:
        target_x = target_y = target_z = 'NULL'
    if injection['actual_coords'] != '':
        i = injection['actual_coords'].split(",")
        actual_x = re.sub(pattern, '', i[0])
        actual_y = re.sub(pattern, '', i[1])
        actual_z = re.sub(pattern, '', i[2])
    else:
        actual_x = actual_y = actual_z = 'NULL'
    query = query + "\nUPDATE injectionSitesLocations set x=" + target_x + ", y=" + target_y + ", z=" + target_z + ", types='target' WHERE injection_sites_id=(SELECT i.id FROM injectionSites i INNER JOIN surgeries s ON i.surgery_id=s.id INNER JOIN tissueDissections td on s.tissue_dissection_id=td.id WHERE td.tissue_code='"+id+"' order by id desc limit "+str(ind)+",1) AND types='target';"
    query = query + "\nUPDATE injectionSitesLocations set x=" + actual_x + ", y=" + actual_y + ", z=" + actual_z + ", types='actual' WHERE injection_sites_id=(SELECT i.id FROM injectionSites i INNER JOIN surgeries s ON i.surgery_id=s.id INNER JOIN tissueDissections td on s.tissue_dissection_id=td.id WHERE td.tissue_code='"+id+"' order by id desc limit "+str(ind)+",1) AND types='actual';"    
    ind = ind - 1

query = query + "\nDROP TRIGGER after_injectionSites_insert;"
f.write(query)
f.close()
